In [19]:
import ast
import math
import torch
import torchvision
import pickle as pkl
import pandas as pd
import numpy as np
import seaborn as sns
from einops import rearrange
import matplotlib.pyplot as plt

from torchmetrics.functional.classification import (
    multilabel_f1_score,
    multilabel_precision,
    multilabel_recall,
)

from data_utils import results2df

In [20]:
train_path = "/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/dataset/results/model=slow_r50_ds=panaf_seq_fd_only_feats=train_feats.pkl"
val_path = "/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/dataset/results/model=slow_r50_ds=panaf_seq_fd_only_e=200_feats=val_feats.pkl"
metadata_path = "/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/dataset/metadata/with_negative_pairing/new_metadata.csv"

In [21]:
with open(train_path, "rb") as f:
    train_data = pkl.load(f)

with open(val_path, "rb") as f:
    val_data = pkl.load(f)

metadata = pd.read_csv(metadata_path)

with open("../dataset/metadata/behaviours.txt", "rb") as f:
    behaviours = [beh.decode("utf-8").strip() for beh in f.readlines()]

In [23]:
def calculate_metrics(preds, labels):
    # Convert preds and labels to tensors
    preds, labels = np.stack(preds), np.stack(labels)
    preds, labels = torch.tensor(preds, dtype=torch.float32), torch.tensor(labels)
    # Calculate metrics
    f1 = multilabel_f1_score(preds, labels, num_labels=14, average="none")
    precision = multilabel_precision(preds, labels, num_labels=14, average="none")
    recall = multilabel_recall(preds, labels, num_labels=14, average="none")
    return f1, precision, recall

In [38]:
def measure_domain_shift(train_df, val_df, pred_column, behaviours):

    store = []

    for idx in range(len(behaviours)):
        val_agg_df = val_df[val_df.label.apply(lambda x: x[idx] == 1)]
        train_agg_df = train_df[train_df.label.apply(lambda x: x[idx] == 1)]

        overall_f1, overall_precision, overall_recall = calculate_metrics(
            val_agg_df[pred_column].values, val_agg_df["label"].values
        )

        mutual_df = val_agg_df[val_agg_df["utm"].isin(train_agg_df["utm"])]
        mutual_videos = len(mutual_df["utm"].unique())

        mutual_f1, mutual_precision, mutual_recall = calculate_metrics(
            mutual_df[pred_column].values, mutual_df["label"].values
        )

        exclusive_df = val_agg_df[~val_agg_df["utm"].isin(train_agg_df["utm"])]
        exclusive_videos = len(exclusive_df["utm"].unique())

        exclusive_f1, exclusive_precision, exclusive_recall = calculate_metrics(
            exclusive_df[pred_column].values, exclusive_df["label"].values
        )

        store.append(
            {
                "behaviour": behaviours[idx],
                "mutual_ct_loc": mutual_videos,
                "exclusive_ct_loc": exclusive_videos,
                "mutual_loc_prop": round(
                    mutual_videos / (exclusive_videos + mutual_videos), 2
                ),
                "overall_recall": overall_recall[idx].item(),
                "overall_precision": overall_precision[idx].item(),
                "overall_f1": overall_f1[idx].item(),
                "mutual_recall": mutual_recall[idx].item(),
                "mutual_precision": mutual_precision[idx].item(),
                "mutual_f1": mutual_f1[idx].item(),
                "exclusive_recall": exclusive_recall[idx].item(),
                "exclusive_precision": exclusive_precision[idx].item(),
                "exclusive_f1": exclusive_f1[idx].item(),
            }
        )

        df = pd.DataFrame(store)

        # Round all numerical columns to 2 decimal places
        df = df.round(4)

    return mutual_df, exclusive_df, df

In [39]:
train_df, val_df = results2df(train_data, val_data, metadata)

/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/notebooks/data_utils.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  df["pred"] = df.pred.apply(lambda x: torch.sigmoid(torch.tensor(x)))


In [40]:
mutual_df, exclusive_df, original_df = measure_domain_shift(
    train_df, val_df, pred_column="pred", behaviours=behaviours
)

In [41]:
original_df

,behaviour,mutual_ct_loc,exclusive_ct_loc,mutual_loc_prop,overall_recall,overall_precision,overall_f1,mutual_recall,mutual_precision,mutual_f1,exclusive_recall,exclusive_precision,exclusive_f1
0,aggression,6,8,0.43,0.2800,1.0,0.4375,0.3529,1.0,0.5217,0.1250,1.0,0.2222
1,bipedal,11,11,0.50,0.3269,1.0,0.4928,0.5000,1.0,0.6667,0.0000,0.0,0.0000
2,camera_reaction,44,39,0.53,0.4676,1.0,0.6373,0.5213,1.0,0.6853,0.3556,1.0,0.5246
3,climbing,35,20,0.64,0.6154,1.0,0.7619,0.6911,1.0,0.8173,0.1500,1.0,0.2609
4,display,7,5,0.58,0.5263,1.0,0.6897,0.6061,1.0,0.7547,0.0000,0.0,0.0000
5,feeding,57,16,0.78,0.7584,1.0,0.8626,0.8088,1.0,0.8943,0.0556,1.0,0.1053
6,grooming,24,16,0.60,0.2609,1.0,0.4138,0.3462,1.0,0.5143,0.0000,0.0,0.0000
7,object_carrying,27,23,0.54,0.4020,1.0,0.5734,0.5333,1.0,0.6957,0.0370,1.0,0.0714
8,piloerection,6,7,0.46,0.2000,1.0,0.3333,0.4286,1.0,0.6000,0.0000,0.0,0.0000
9,playing,3,6,0.33,0.0000,0.0,0.0000,0.0000,0.0,0.0000,0.0000,0.0,0.0000


In [32]:
original_df[
    ["behaviour", "mutual_loc_prop", "overall_f1", "mutual_f1", "exclusive_f1"]
].sort_values(by="mutual_loc_prop", ascending=False)

,behaviour,mutual_loc_prop,overall_f1,mutual_f1,exclusive_f1
11,tool_use,0.86,0.9180,0.9379,0.3333
5,feeding,0.78,0.8626,0.8943,0.1053
12,travel,0.78,0.8767,0.8713,0.9147
10,resting,0.72,0.8416,0.8627,0.6176
3,climbing,0.64,0.7619,0.8173,0.2609
6,grooming,0.60,0.4138,0.5143,0.0000
4,display,0.58,0.6897,0.7547,0.0000
7,object_carrying,0.54,0.5734,0.6957,0.0714
2,camera_reaction,0.53,0.6373,0.6853,0.5246
1,bipedal,0.50,0.4928,0.6667,0.0000


In [36]:
# Calculate correlation between mutual_loc_prop and overall_f1
overall_corr = original_df[["mutual_loc_prop", "overall_f1"]].corr().iloc[0, 1]
mutual_corr = original_df[["mutual_loc_prop", "mutual_f1"]].corr().iloc[0, 1]
exclusive_corr = original_df[["mutual_loc_prop", "exclusive_f1"]].corr().iloc[0, 1]

print(f"Overall correlation: {overall_corr}")
print(f"Mutual correlation: {mutual_corr}")
print(f"Exclusive correlation: {exclusive_corr}")

Overall correlation: 0.9063299634892453
Mutual correlation: 0.8211864131686705
Exclusive correlation: 0.5554852085365047
